<a href="https://colab.research.google.com/github/duvvusuribabu/NLP/blob/main/LanguageModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Importing the required libraries**

In [1]:
import pandas as pd
import re
import random
import tarfile
import os
import math

### **Connecting google drive with notebook**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### **Extracting data from the tar file which was downloaded from the https://devopedia.org/text-corpus-for-nlp.**

### **To extract data from tarfile we've imported the module `tar`**

In [3]:
from google.colab import drive
drive.mount('/content/drive')
import tarfile
fname = "/content/drive/MyDrive/aclImdb_v1.tar.gz"
tar = tarfile.open(fname, "r:gz")
tar.extractall('.')
tar.close()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **Creaintg corpus from downloaded data**

In [4]:
reviews = []
def adder(file):
  for i in os.listdir(file):
    fname = os.path.join(file,i)
    f = open(fname,'r')
    reviews.append(f.read().split("."))

adder("/content/aclImdb/train/pos")
adder("/content/aclImdb/train/neg")

In [5]:
len(reviews) #total no of reviews

25000

### **Size of the corpus**

In [6]:
data = []
for i in reviews:
  for j in i:
    if j!="":
      data.append(j)
len(data)

305571

In [7]:
data[60]

' In solving one set of problems, Guiness has created dozens of others'

### **Removing punctuations and additional html tags from the corpus(i.e.,cleaning data)**

In [8]:
reg = "<br /><br />|\'s|\.\.+|\!|\@|\#|\$|\%|\^|\&|\*|\(|\)|\_|\-|\=|\+|\}|\{|\[|\]|\:|\;|\'|\"|\>|\<|\,|\/|\`|\~"
def tagremover(sent):
  sent = re.sub(reg," ",sent)
  return sent

In [9]:
for i in range(len(data)):
  data[i] = tagremover(data[i]).lower().strip()

In [10]:
data

['a super  unusual film from audiard  read my lips is a pulpy  lonely  hearts thriller',
 'it  perfect for the handsomely grizzled charisma of vincent cassel and features a marvellously contained performance from emanuelle devos',
 'devos is a recurring feature of audiard in the same way that karin viard pops up for jean pierre jeunet  unconventionally beautiful  she  referred to by everyone as unattractive in this film   versatile and capable of a subordinate profile',
 'this is almost the definition of her role as carla  a put upon office dogsbody  taunted by colleagues exploiting her deafness',
 'yet she finds an ami d exploitation  if you like in cassel  ex con paul',
 'each exploits the other  unconventional talents  theft and lip reading  to struggle through their respective situations and form an unconventionally romantic rapprochement',
 'devos audiard manage carla  deafness and its attendant  warped inner world with discreet  stylish flair',
 'in this film  2001  audiard is al

In [11]:
from tqdm import tqdm

### **Creating Ngram model(i.e., finding ngram probabilities)**

In [12]:
class N_gram:
  def __init__(self,data,n):
    self.data = data
    self.n = n
  def gramDiv(self):
    list_of_ngrams=[]
    gram_count = dict()
    hist_count = dict()
    sent_data=[]
    if n==1:
      for i in tqdm(range(len(self.data))):
        sentence = data[i].split()
        for j in range(len(sentence)):
          current_word = sentence[j]
          list_of_ngrams.append((current_word))
          gram_count[(current_word)] = gram_count.get((current_word), 0)+1
          hist_count[(current_word)] = hist_count.get(((current_word)),0)+1
    else:
      for i in tqdm(range(len(self.data))):
        sent = "<s> "*(self.n-1)+self.data[i]+" </s>"
        sent_data.append(sent)
        
        sentence = sent.split()
        given_n_word = sentence[:self.n-1]
        for j in range(self.n-1,len(sentence)):
          current_word = sentence[j]
          list_of_ngrams.append((tuple(given_n_word), current_word))
          gram_count[(tuple(given_n_word), current_word)] = gram_count.get((tuple(given_n_word), current_word), 0)+1
          hist_count[tuple(given_n_word)] = hist_count.get(tuple(given_n_word),0)+1 
          given_n_word.pop(0)
          given_n_word +=[current_word]
    return gram_count,hist_count,list_of_ngrams,sent_data


In [13]:
n=int(input("Enter n-value:"))
obj = N_gram(data,n)
gram_n1_count,gram_n0_count,list_of_ngrams,sent_data= obj.gramDiv()

Enter n-value:3


100%|██████████| 305571/305571 [00:23<00:00, 13112.71it/s]


In [14]:
def word_count_in_corpus(sent_data):
  word_count_corpus=dict()
  for i in tqdm(range(len(sent_data))):
    for j in sent_data[i].split():
      word_count_corpus[j]=word_count_corpus.get(j,0)+1
  return word_count_corpus

In [15]:
word_count_corpus= word_count_in_corpus(sent_data)

100%|██████████| 305571/305571 [00:01<00:00, 167004.63it/s]


In [16]:
word_count_corpus

{'<s>': 611142,
 'a': 163108,
 'super': 497,
 'unusual': 309,
 'film': 39967,
 'from': 20462,
 'audiard': 25,
 'read': 1963,
 'my': 12503,
 'lips': 106,
 'is': 107288,
 'pulpy': 7,
 'lonely': 188,
 'hearts': 135,
 'thriller': 888,
 '</s>': 305571,
 'it': 96016,
 'perfect': 1597,
 'for': 44296,
 'the': 336718,
 'handsomely': 8,
 'grizzled': 19,
 'charisma': 138,
 'of': 145844,
 'vincent': 144,
 'cassel': 31,
 'and': 164117,
 'features': 641,
 'marvellously': 6,
 'contained': 113,
 'performance': 2890,
 'emanuelle': 6,
 'devos': 26,
 'recurring': 36,
 'feature': 788,
 'in': 93945,
 'same': 4048,
 'way': 7993,
 'that': 73061,
 'karin': 12,
 'viard': 1,
 'pops': 89,
 'up': 13251,
 'jean': 351,
 'pierre': 57,
 'jeunet': 7,
 'unconventionally': 4,
 'beautiful': 2175,
 'she': 14205,
 'referred': 74,
 'to': 135700,
 'by': 22545,
 'everyone': 2219,
 'as': 46932,
 'unattractive': 59,
 'this': 75791,
 'versatile': 35,
 'capable': 228,
 'subordinate': 5,
 'profile': 47,
 'almost': 3139,
 'definiti

In [17]:
len(word_count_corpus) #vocabulary size i.e., unique words in corpus

80052

In [18]:
gram_n1_count

{(('<s>', '<s>'), 'a'): 4790,
 (('<s>', 'a'), 'super'): 6,
 (('a', 'super'), 'unusual'): 1,
 (('super', 'unusual'), 'film'): 1,
 (('unusual', 'film'), 'from'): 2,
 (('film', 'from'), 'audiard'): 1,
 (('from', 'audiard'), 'read'): 1,
 (('audiard', 'read'), 'my'): 2,
 (('read', 'my'), 'lips'): 18,
 (('my', 'lips'), 'is'): 5,
 (('lips', 'is'), 'a'): 4,
 (('is', 'a'), 'pulpy'): 2,
 (('a', 'pulpy'), 'lonely'): 1,
 (('pulpy', 'lonely'), 'hearts'): 1,
 (('lonely', 'hearts'), 'thriller'): 1,
 (('hearts', 'thriller'), '</s>'): 1,
 (('<s>', '<s>'), 'it'): 17298,
 (('<s>', 'it'), 'perfect'): 4,
 (('it', 'perfect'), 'for'): 8,
 (('perfect', 'for'), 'the'): 44,
 (('for', 'the'), 'handsomely'): 1,
 (('the', 'handsomely'), 'grizzled'): 1,
 (('handsomely', 'grizzled'), 'charisma'): 1,
 (('grizzled', 'charisma'), 'of'): 1,
 (('charisma', 'of'), 'vincent'): 1,
 (('of', 'vincent'), 'cassel'): 1,
 (('vincent', 'cassel'), 'and'): 2,
 (('cassel', 'and'), 'features'): 1,
 (('and', 'features'), 'a'): 6,
 (('f

In [19]:
gram_n0_count

{('<s>', '<s>'): 305571,
 ('<s>', 'a'): 4790,
 ('a', 'super'): 90,
 ('super', 'unusual'): 1,
 ('unusual', 'film'): 11,
 ('film', 'from'): 253,
 ('from', 'audiard'): 1,
 ('audiard', 'read'): 2,
 ('read', 'my'): 36,
 ('my', 'lips'): 20,
 ('lips', 'is'): 5,
 ('is', 'a'): 13109,
 ('a', 'pulpy'): 3,
 ('pulpy', 'lonely'): 1,
 ('lonely', 'hearts'): 1,
 ('hearts', 'thriller'): 1,
 ('<s>', 'it'): 17298,
 ('it', 'perfect'): 12,
 ('perfect', 'for'): 126,
 ('for', 'the'): 7963,
 ('the', 'handsomely'): 1,
 ('handsomely', 'grizzled'): 1,
 ('grizzled', 'charisma'): 1,
 ('charisma', 'of'): 13,
 ('of', 'vincent'): 5,
 ('vincent', 'cassel'): 18,
 ('cassel', 'and'): 5,
 ('and', 'features'): 29,
 ('features', 'a'): 86,
 ('a', 'marvellously'): 2,
 ('marvellously', 'contained'): 1,
 ('contained', 'performance'): 1,
 ('performance', 'from'): 89,
 ('from', 'emanuelle'): 1,
 ('emanuelle', 'devos'): 2,
 ('<s>', 'devos'): 4,
 ('devos', 'is'): 5,
 ('a', 'recurring'): 8,
 ('recurring', 'feature'): 1,
 ('feature', 

In [20]:
list_of_ngrams

[(('<s>', '<s>'), 'a'),
 (('<s>', 'a'), 'super'),
 (('a', 'super'), 'unusual'),
 (('super', 'unusual'), 'film'),
 (('unusual', 'film'), 'from'),
 (('film', 'from'), 'audiard'),
 (('from', 'audiard'), 'read'),
 (('audiard', 'read'), 'my'),
 (('read', 'my'), 'lips'),
 (('my', 'lips'), 'is'),
 (('lips', 'is'), 'a'),
 (('is', 'a'), 'pulpy'),
 (('a', 'pulpy'), 'lonely'),
 (('pulpy', 'lonely'), 'hearts'),
 (('lonely', 'hearts'), 'thriller'),
 (('hearts', 'thriller'), '</s>'),
 (('<s>', '<s>'), 'it'),
 (('<s>', 'it'), 'perfect'),
 (('it', 'perfect'), 'for'),
 (('perfect', 'for'), 'the'),
 (('for', 'the'), 'handsomely'),
 (('the', 'handsomely'), 'grizzled'),
 (('handsomely', 'grizzled'), 'charisma'),
 (('grizzled', 'charisma'), 'of'),
 (('charisma', 'of'), 'vincent'),
 (('of', 'vincent'), 'cassel'),
 (('vincent', 'cassel'), 'and'),
 (('cassel', 'and'), 'features'),
 (('and', 'features'), 'a'),
 (('features', 'a'), 'marvellously'),
 (('a', 'marvellously'), 'contained'),
 (('marvellously', 'cont

In [21]:
sent_data

['<s> <s> a super  unusual film from audiard  read my lips is a pulpy  lonely  hearts thriller </s>',
 '<s> <s> it  perfect for the handsomely grizzled charisma of vincent cassel and features a marvellously contained performance from emanuelle devos </s>',
 '<s> <s> devos is a recurring feature of audiard in the same way that karin viard pops up for jean pierre jeunet  unconventionally beautiful  she  referred to by everyone as unattractive in this film   versatile and capable of a subordinate profile </s>',
 '<s> <s> this is almost the definition of her role as carla  a put upon office dogsbody  taunted by colleagues exploiting her deafness </s>',
 '<s> <s> yet she finds an ami d exploitation  if you like in cassel  ex con paul </s>',
 '<s> <s> each exploits the other  unconventional talents  theft and lip reading  to struggle through their respective situations and form an unconventionally romantic rapprochement </s>',
 '<s> <s> devos audiard manage carla  deafness and its attendant 

### **Finding probabilities**

In [22]:
def calculate_prob(gram_n0_count,gram_n1_count,list_of_ngrams):
  ngram_probs=dict()
  for i in list_of_ngrams:
    numerator=gram_n1_count.get(i)
    denominator=gram_n0_count.get(i[0])
    ngram_probs[i]=numerator/denominator
  return ngram_probs

In [23]:
ngram_probs=calculate_prob(gram_n0_count,gram_n1_count,list_of_ngrams)

In [24]:
data[0]

'a super  unusual film from audiard  read my lips is a pulpy  lonely  hearts thriller'

In [25]:
ngram_probs #probabilities of ngrams

{(('<s>', '<s>'), 'a'): 0.01567557130748664,
 (('<s>', 'a'), 'super'): 0.0012526096033402922,
 (('a', 'super'), 'unusual'): 0.011111111111111112,
 (('super', 'unusual'), 'film'): 1.0,
 (('unusual', 'film'), 'from'): 0.18181818181818182,
 (('film', 'from'), 'audiard'): 0.003952569169960474,
 (('from', 'audiard'), 'read'): 1.0,
 (('audiard', 'read'), 'my'): 1.0,
 (('read', 'my'), 'lips'): 0.5,
 (('my', 'lips'), 'is'): 0.25,
 (('lips', 'is'), 'a'): 0.8,
 (('is', 'a'), 'pulpy'): 0.0001525669387443741,
 (('a', 'pulpy'), 'lonely'): 0.3333333333333333,
 (('pulpy', 'lonely'), 'hearts'): 1.0,
 (('lonely', 'hearts'), 'thriller'): 1.0,
 (('hearts', 'thriller'), '</s>'): 1.0,
 (('<s>', '<s>'), 'it'): 0.05660877504737033,
 (('<s>', 'it'), 'perfect'): 0.00023124060585038732,
 (('it', 'perfect'), 'for'): 0.6666666666666666,
 (('perfect', 'for'), 'the'): 0.3492063492063492,
 (('for', 'the'), 'handsomely'): 0.0001255808112520407,
 (('the', 'handsomely'), 'grizzled'): 1.0,
 (('handsomely', 'grizzled'), 

In [26]:
test_data= []
def adder(file):
  for i in os.listdir(file):
    fname = os.path.join(file,i)
    f = open(fname,'r')
    test_data.append(f.read().split("."))
adder("/content/aclImdb/test/neg")

### **Importing `secrets` module of python for generating random text sentence from testing data**

In [27]:
import secrets

In [28]:
test_sentence=secrets.choice(secrets.choice(test_data))

In [29]:
test_sentence

' <br /><br />The movie starts out slow and remains that way for about the first thirty minutes, then the suspense part kicks in and starts keeping you a little on edge throughout the rest of the movie'

In [30]:
test_sentence=tagremover(test_sentence).lower().strip()

In [31]:
test_sentence

'the movie starts out slow and remains that way for about the first thirty minutes  then the suspense part kicks in and starts keeping you a little on edge throughout the rest of the movie'

### **Finding probabilities for test sentence**

In [32]:
def prob_for_test_sent(test_sentence,n):
  list_of_ngrams_of_sent=[]
  sent = "<s> "*(n-1)+test_sentence+" </s>"
  sentence_list = sent.split()
  given_n_word = sentence_list[:n-1]
  for j in range(n-1,len(sentence_list)):
    current_word = sentence_list[j]
    list_of_ngrams_of_sent.append((tuple(given_n_word), current_word))
    given_n_word.pop(0)
    given_n_word +=[current_word]
  return list_of_ngrams_of_sent,sentence_list

In [33]:
list_of_ngrams_of_sent,sentence_list=prob_for_test_sent(test_sentence,n)

In [34]:
list_of_ngrams_of_sent

[(('<s>', '<s>'), 'the'),
 (('<s>', 'the'), 'movie'),
 (('the', 'movie'), 'starts'),
 (('movie', 'starts'), 'out'),
 (('starts', 'out'), 'slow'),
 (('out', 'slow'), 'and'),
 (('slow', 'and'), 'remains'),
 (('and', 'remains'), 'that'),
 (('remains', 'that'), 'way'),
 (('that', 'way'), 'for'),
 (('way', 'for'), 'about'),
 (('for', 'about'), 'the'),
 (('about', 'the'), 'first'),
 (('the', 'first'), 'thirty'),
 (('first', 'thirty'), 'minutes'),
 (('thirty', 'minutes'), 'then'),
 (('minutes', 'then'), 'the'),
 (('then', 'the'), 'suspense'),
 (('the', 'suspense'), 'part'),
 (('suspense', 'part'), 'kicks'),
 (('part', 'kicks'), 'in'),
 (('kicks', 'in'), 'and'),
 (('in', 'and'), 'starts'),
 (('and', 'starts'), 'keeping'),
 (('starts', 'keeping'), 'you'),
 (('keeping', 'you'), 'a'),
 (('you', 'a'), 'little'),
 (('a', 'little'), 'on'),
 (('little', 'on'), 'edge'),
 (('on', 'edge'), 'throughout'),
 (('edge', 'throughout'), 'the'),
 (('throughout', 'the'), 'rest'),
 (('the', 'rest'), 'of'),
 (('re

In [35]:
sentence_list

['<s>',
 '<s>',
 'the',
 'movie',
 'starts',
 'out',
 'slow',
 'and',
 'remains',
 'that',
 'way',
 'for',
 'about',
 'the',
 'first',
 'thirty',
 'minutes',
 'then',
 'the',
 'suspense',
 'part',
 'kicks',
 'in',
 'and',
 'starts',
 'keeping',
 'you',
 'a',
 'little',
 'on',
 'edge',
 'throughout',
 'the',
 'rest',
 'of',
 'the',
 'movie',
 '</s>']

### **Finding vocabulary size**

In [36]:
def calc_of_V(sentence_list,word_count_corpus):
  v=0
  for i in range(len(sentence_list)):
    if sentence_list[i] in word_count_corpus:
      v+=word_count_corpus.get(sentence_list[i])
  return v

In [37]:
v=calc_of_V(sentence_list,word_count_corpus)

In [38]:
v

4297573

In [39]:
list_of_ngrams_of_sent

[(('<s>', '<s>'), 'the'),
 (('<s>', 'the'), 'movie'),
 (('the', 'movie'), 'starts'),
 (('movie', 'starts'), 'out'),
 (('starts', 'out'), 'slow'),
 (('out', 'slow'), 'and'),
 (('slow', 'and'), 'remains'),
 (('and', 'remains'), 'that'),
 (('remains', 'that'), 'way'),
 (('that', 'way'), 'for'),
 (('way', 'for'), 'about'),
 (('for', 'about'), 'the'),
 (('about', 'the'), 'first'),
 (('the', 'first'), 'thirty'),
 (('first', 'thirty'), 'minutes'),
 (('thirty', 'minutes'), 'then'),
 (('minutes', 'then'), 'the'),
 (('then', 'the'), 'suspense'),
 (('the', 'suspense'), 'part'),
 (('suspense', 'part'), 'kicks'),
 (('part', 'kicks'), 'in'),
 (('kicks', 'in'), 'and'),
 (('in', 'and'), 'starts'),
 (('and', 'starts'), 'keeping'),
 (('starts', 'keeping'), 'you'),
 (('keeping', 'you'), 'a'),
 (('you', 'a'), 'little'),
 (('a', 'little'), 'on'),
 (('little', 'on'), 'edge'),
 (('on', 'edge'), 'throughout'),
 (('edge', 'throughout'), 'the'),
 (('throughout', 'the'), 'rest'),
 (('the', 'rest'), 'of'),
 (('re

In [40]:
list_of_ngrams_of_sent[0][1]

'the'

### **Finding `perplexity` for the test sentence**

In [41]:
def perplexity_of_test_sent(list_of_ngrams_of_sent,n,v):
  sent_prob=1
  for j in list_of_ngrams_of_sent:
    given_n_word=j[0]
    current_word=j[1]
    if ((given_n_word, current_word)) in gram_n1_count.keys():
      numerator1=gram_n1_count.get((tuple(given_n_word), current_word))+1
    if ((given_n_word, current_word)) not in gram_n1_count.keys():
      numerator1=0+1
    if given_n_word in gram_n0_count.keys():
      denominator1=gram_n0_count.get(tuple(given_n_word))+v
    if given_n_word not in gram_n0_count.keys():
      denominator1=0+v
    probobality=numerator1/denominator1
    sent_prob*=probobality
  
  return sent_prob

In [42]:
sent_prob=perplexity_of_test_sent(list_of_ngrams_of_sent,n,v)

In [43]:
import math
print("perplexity of the test sentence in with respect to corpus")
math.pow(sent_prob,(1/len(sentence_list))) # perplexity of test sentence

perplexity of the test sentence in with respect to corpus


3.927447151611063e-06